# Tarea 7: Aplicación de perceptrón multicapa
*Gabriel Alejandro Morales Ruiz*

## Enunciado del problema
En la tabla adjunta se muestran los datos de 1000 clientes que solicitaron créditos a un banco dado. La última columna muestra la información de los clientes que cayeron en mora en algún momento del período del crédito. El monto máximo de crédito que puede asignarse son $300,000 y la antigüedad laboral máxima que se toma en cuenta para asignar el crédito es de 15 años (es decir, antigüedades mayores ya no generan más posibilidad de ser aprobado).

Se busca una relación entre la información presentada (que se obtiene al contratar el crédito) y la posibilidad de que el cliente caiga en mora en algún momento del plazo.

Entrene un perceptrón multicapa para encontrar una relación tomando como entradas el monto solicitado (normalizado), la carga que implica al salario el pago de la mensualidad y la antigüedad laboral al contratar (normalizada).

Utilice 70-30 de relación entrenamiento-prueba y calcule el accuracy.

El número de neuronas ocultas es a su criterio.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Código para entrenar redes
def f(x, alpha) :
    return 1 / (1 + np.exp(-alpha*x))

def n_significant_figures(num, n) :
    return float(format(num, '.'+str(n)+'g'))

def train_NN(x, d, alpha, L = None) :
    Q = np.size(x, axis=0)
    N = np.size(x, axis=1)
    M = np.size(d, axis=1)
    if L == None : L = N + M
    w_h = np.random.uniform(-1, 1, (L, N))
    w_o = np.random.uniform(-1, 1, (M, L))
    
    error = np.Inf
    previous_error = 0
    vf = np.vectorize(f)
    
    while(error != previous_error) :
        for j in np.arange(Q) :
            net_h = w_h @ x[j].T
            y_h = vf(net_h, alpha)            
            net_o = w_o @ y_h
            y = vf(net_o, alpha)
    
            delta_o = np.multiply((d[j].T - y), np.multiply(y, (1-y)))
            delta_h = np.multiply((delta_o.T @ w_o).T, np.multiply(y_h, 1-y_h))
            w_h = w_h + alpha*delta_h @ x[j]
            w_o = w_o + alpha*delta_o @ (y_h.T) 
        
        previous_error = error
        error = n_significant_figures(np.linalg.norm(delta_o), 5)
    return w_h, w_o, error

def predict(inputs, w_h, w_o, alpha) :
    vf = np.vectorize(f)
    
    net_h = w_h @ inputs.T
    y_h = vf(net_h, alpha)            
    net_o = w_o @ y_h
    y = vf(net_o, alpha)

    return y

In [2]:
# Adecuando los datos
data = pd.read_excel("PercMultAplicado (1).xlsx")

# Monto máximo del préstamo es 300k
data["Monto"] /= 300000

# Carga definida como porcentaje del pago sobre el ingreso mensual
data["Carga"] = data["Mensualidad"]/data["Ingreso mensual"]

# Antigüedad máxima tomada como 15 años (12*15=180)
data[data["Antigüedad laboral (meses)"] > 180] = 180
data["Antigüedad laboral (meses)"] /= 180

# Mapeo de variable de salida a binario
data["Mora"] = data["Mora"].map({"SI": 1, "NO": 0})

# Recorte de columnas no usadas
X = data[["Monto", "Carga", "Antigüedad laboral (meses)", "Mora"]]
print(X)

        Monto     Carga  Antigüedad laboral (meses)  Mora
0    0.997333  0.185128                    0.322222     1
1    0.937000  0.083111                    0.827778     0
2    0.896000  0.270089                    0.744444     0
3    0.855333  0.178150                    0.427778     0
4    0.855000  0.027296                    0.027778     1
..        ...       ...                         ...   ...
995  0.062333  0.165289                    0.833333     0
996  0.055333  0.114986                    0.138889     1
997  0.054667  0.154361                    0.594444     0
998  0.048667  0.219692                    0.333333     0
999  0.045000  0.305333                    0.383333     0

[1000 rows x 4 columns]


In [3]:
# Revolver muestras
X = X.sample(frac=1)
d = np.matrix(X.iloc[:, -1]).T
X = np.matrix(X.iloc[:, 0:-1])

# Porcentaje utilizado para entrenar (el resto se usa para validar)
perc = 0.7
m = len(d)
cutoff = round(m*perc)

# Separar
X_train = X[0:cutoff, :]
d_train = d[0:cutoff, :]

X_val = X[cutoff:, :]
d_val = d[cutoff:, :]

In [4]:
# Revolviendo muestras y separando
alpha = 2

w_h, w_o, error = train_NN(X_train, d, alpha)

In [5]:
vround = np.vectorize(round)
errors = 0
for i in range(len(d_val)) :
    result = predict(X_val[i], w_h, w_o, alpha)
    delta_o = np.multiply((d_val[i].T - result), np.multiply(result, (1-result)))
    result = vround(result)
    if result != d_val[i] :
        errors += 1
        print("Not Ok. delta_o = " + str(delta_o))
print("Misclassified " + str(errors))
accuracy = 100 - 100*(errors)/(m - cutoff)
print("Accuracy: " + str(accuracy) + "%")

Not Ok. delta_o = [[0.14237138]]
Not Ok. delta_o = [[0.1413435]]
Not Ok. delta_o = [[0.1410368]]
Not Ok. delta_o = [[0.13701204]]
Not Ok. delta_o = [[0.14285038]]
Not Ok. delta_o = [[0.12188198]]
Not Ok. delta_o = [[0.13697729]]
Not Ok. delta_o = [[0.12883415]]
Misclassified 8
Accuracy: 97.33333333333333%
